In [1]:
/*
    Created by Indra Mahkota
*/

%use @gson.json
%use krangl
%use coroutines
%use @retrofit.json
%use @retrofit_gson_converter.json
%use @org_json.json

import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.flow.MutableStateFlow
import kotlinx.coroutines.flow.StateFlow
import kotlinx.coroutines.flow.collect
import kotlinx.coroutines.GlobalScope
import kotlinx.coroutines.launch

In [2]:
data class UserResponse(
    @SerializedName("incomplete_results")
    val incompleteResults: Boolean?,
    @SerializedName("items")
    val items: List<UserDto>,
    @SerializedName("total_count")
    val totalCount: Long?
)

data class UserDto(
    @SerializedName("id")
    val id: Long,
    @SerializedName("avatar_url")
    val avatarUrl: String?,
    @SerializedName("name")
    val name: String?,
    @SerializedName("login")
    val login: String?,
    @SerializedName("bio")
    val bio: String?,
    @SerializedName("followers")
    val followers: Int?,
    @SerializedName("following")
    val following: Int?,
    @SerializedName("location")
    val location: String?,
    @SerializedName("email")
    val email: String?
)

data class User(
    val id: Long,
    val avatarUrl: String,
    val name: String,
    val login: String,
    val bio: String,
    val followers: Int,
    val following: Int,
    val location: String,
    val email: String
)

In [3]:
fun UserDto.toUser() = User(
    id = id,
    avatarUrl = avatarUrl ?: "",
    name = name ?: "N/A",
    login = login ?: "N/A",
    bio = bio ?: "Biography not available",
    followers = followers ?: 0,
    following = following ?: 0,
    location = location ?: "N/A",
    email = email ?: "N/A"
)

fun List<UserDto>.toListUser() =
    this.map {
        it.toUser()
    }

In [4]:
interface GithubService {
    @JvmSuppressWildcards
    @GET("search/users")
    suspend fun searchUsers(
        @QueryMap params: Map<String, Any>
    ): Response<UserResponse>

    @GET("users/{id}")
    suspend fun getUser(
        @Path("id") username: String
    ): Response<UserDto>
}

In [5]:
val retrofit = Retrofit
        .Builder().apply {
            baseUrl("https://api.github.com/")
            addConverterFactory(GsonConverterFactory.create(GsonBuilder().create()))
        }.build()
        
val service = retrofit.create(GithubService::class.java)

In [14]:
val indramahkota = "{\"login\":\"indramahkota\",\"id\":34052126,\"node_id\":\"MDQ6VXNlcjM0MDUyMTI2\",\"avatar_url\":\"https://avatars.githubusercontent.com/u/34052126?v=4\",\"gravatar_id\":\"\",\"url\":\"https://api.github.com/users/indramahkota\",\"html_url\":\"https://github.com/indramahkota\",\"followers_url\":\"https://api.github.com/users/indramahkota/followers\",\"following_url\":\"https://api.github.com/users/indramahkota/following{/other_user}\",\"gists_url\":\"https://api.github.com/users/indramahkota/gists{/gist_id}\",\"starred_url\":\"https://api.github.com/users/indramahkota/starred{/owner}{/repo}\",\"subscriptions_url\":\"https://api.github.com/users/indramahkota/subscriptions\",\"organizations_url\":\"https://api.github.com/users/indramahkota/orgs\",\"repos_url\":\"https://api.github.com/users/indramahkota/repos\",\"events_url\":\"https://api.github.com/users/indramahkota/events{/privacy}\",\"received_events_url\":\"https://api.github.com/users/indramahkota/received_events\",\"type\":\"User\",\"site_admin\":false,\"name\":\"Indra Mahkota\",\"company\":null,\"blog\":\"indramahkota.info\",\"location\":\"Pontianak, Indonesia\",\"email\":null,\"hireable\":true,\"bio\":\"Focus on developing android applications and having some knowledge of web frontend technologies.\uD83D\uDE0A.\",\"twitter_username\":null,\"public_repos\":54,\"public_gists\":9,\"followers\":20,\"following\":19,\"created_at\":\"2017-11-28T05:26:02Z\",\"updated_at\":\"2022-01-13T07:58:02Z\"}"

In [15]:
val inDto = Gson().fromJson(indramahkota, UserDto::class.java)
val inU = inDto.toUser()

inU

User(id=34052126, avatarUrl=https://avatars.githubusercontent.com/u/34052126?v=4, name=Indra Mahkota, login=indramahkota, bio=Focus on developing android applications and having some knowledge of web frontend technologies.😊., followers=20, following=19, location=Pontianak, Indonesia, email=N/A)

In [6]:
suspend fun searchUsers(params: Map<String, Any>): UserResponse? {
    return try {
        val response = service.searchUsers(params)
        if(response.isSuccessful)
            response.body()
        else null
    } catch (e: Exception) {
        println(e.message ?: "Unknown error")
        null
    }
}

suspend fun getUser(username: String): UserDto? {
    return try {
        val response = service.getUser(username)
        if(response.isSuccessful)
            response.body()
        else null
    } catch (e: Exception) {
        println(e.message ?: "Unknown error")
        null
    }
}

val listOfUser = mutableListOf<User>()
val listUserDto = mutableListOf<UserDto>()

fun searchUsersFunction() = runBlocking {
    val response = searchUsers(mapOf("q" to "josh", "per_page" to 10))
    
    response?.items?.forEach {
        val userDtoResponse = it.login?.let { username ->
            getUser(username)
        }
        if (userDtoResponse != null) {
            listUserDto.add(userDtoResponse)
        }
    }
    
    listOfUser.addAll(listUserDto.toListUser())
    
    println(response?.items?.size.toString())
    println()
    println("result dto: ${response.toString()}")
    println()
    println("result model: ${listOfUser.toString()}")
}

searchUsersFunction()

10

result dto: UserResponse(incompleteResults=false, items=[UserDto(id=137, avatarUrl=https://avatars.githubusercontent.com/u/137?v=4, name=null, login=josh, bio=null, followers=null, following=null, location=null, email=null), UserDto(id=54473, avatarUrl=https://avatars.githubusercontent.com/u/54473?v=4, name=null, login=joshlong, bio=null, followers=null, following=null, location=null, email=null), UserDto(id=6692932, avatarUrl=https://avatars.githubusercontent.com/u/6692932?v=4, name=null, login=joshwcomeau, bio=null, followers=null, following=null, location=null, email=null), UserDto(id=13760, avatarUrl=https://avatars.githubusercontent.com/u/13760?v=4, name=null, login=joshaber, bio=null, followers=null, following=null, location=null, email=null), UserDto(id=10632, avatarUrl=https://avatars.githubusercontent.com/u/10632?v=4, name=null, login=goatslacker, bio=null, followers=null, following=null, location=null, email=null), UserDto(id=3272748, avatarUrl=https://avatars.githubuserc

In [7]:
val listOfUser2 = mutableListOf<User>()
val listUserDto2 = mutableListOf<UserDto>()

fun searchUsersFunction2() = runBlocking {
    val response = searchUsers(mapOf("q" to "josh"))
    
    response?.items?.forEach {
        val userDtoResponse = it.login?.let { username ->
            getUser(username)
        }
        if (userDtoResponse != null) {
            listUserDto2.add(userDtoResponse)
        }
    }
    
    listOfUser.addAll(listUserDto2.toListUser())
    
    println(response?.items?.size.toString())
    println()
    println("result dto: ${listUserDto2.toString()}")
    println()
    println("result model: ${listOfUser.toString()}")
}

searchUsersFunction2()

30

result dto: [UserDto(id=137, avatarUrl=https://avatars.githubusercontent.com/u/137?v=4, name=Joshua Peek, login=josh, bio=null, followers=2351, following=184, location=San Francisco, CA, email=null), UserDto(id=54473, avatarUrl=https://avatars.githubusercontent.com/u/54473?v=4, name=Josh Long, login=joshlong, bio=Josh (@starbuxman) is the Spring Developer Advocate at VMware, an open-source hacker, book/video author and speaker, followers=7955, following=65, location=San Francisco, CA, email=null), UserDto(id=6692932, avatarUrl=https://avatars.githubusercontent.com/u/6692932?v=4, name=Joshua Comeau, login=joshwcomeau, bio=Software developer.
Makes stuff., followers=3578, following=25, location=Montreal, QC, email=null), UserDto(id=13760, avatarUrl=https://avatars.githubusercontent.com/u/13760?v=4, name=Josh Abernathy, login=joshaber, bio=null, followers=2066, following=39, location=The Gem City, email=null), UserDto(id=10632, avatarUrl=https://avatars.githubusercontent.com/u/10632?v

Makes stuff., followers=3578, following=25, location=Montreal, QC, email=N/A), User(id=13760, avatarUrl=https://avatars.githubusercontent.com/u/13760?v=4, name=Josh Abernathy, login=joshaber, bio=Biography not available, followers=2066, following=39, location=The Gem City, email=N/A), User(id=10632, avatarUrl=https://avatars.githubusercontent.com/u/10632?v=4, name=Josh Perez, login=goatslacker, bio=Biography not available, followers=798, following=127, location=Florida, email=N/A), User(id=3272748, avatarUrl=https://avatars.githubusercontent.com/u/3272748?v=4, name=Captain Pangyo, login=joshua1988, bio=Biography not available, followers=1194, following=125, location=South Korea, email=N/A), User(id=3231, avatarUrl=https://avatars.githubusercontent.com/u/3231?v=4, name=josh rotenberg, login=joshrotenberg, bio=hi, i'm josh., followers=44, following=81, location=albany, ca, email=N/A), User(id=3335181, avatarUrl=https://avatars.githubusercontent.com/u/3335181?v=4, name=Josh Goldberg, logi

Makes stuff., followers=3578, following=25, location=Montreal, QC, email=N/A), User(id=13760, avatarUrl=https://avatars.githubusercontent.com/u/13760?v=4, name=Josh Abernathy, login=joshaber, bio=Biography not available, followers=2066, following=39, location=The Gem City, email=N/A), User(id=10632, avatarUrl=https://avatars.githubusercontent.com/u/10632?v=4, name=Josh Perez, login=goatslacker, bio=Biography not available, followers=798, following=127, location=Florida, email=N/A), User(id=3272748, avatarUrl=https://avatars.githubusercontent.com/u/3272748?v=4, name=Captain Pangyo, login=joshua1988, bio=Biography not available, followers=1194, following=125, location=South Korea, email=N/A), User(id=3231, avatarUrl=https://avatars.githubusercontent.com/u/3231?v=4, name=josh rotenberg, login=joshrotenberg, bio=hi, i'm josh., followers=44, following=81, location=albany, ca, email=N/A), User(id=3335181, avatarUrl=https://avatars.githubusercontent.com/u/3335181?v=4, name=Josh Goldberg, logi

In [8]:
val userDtoJsonString = Gson().toJson(listUserDto2)
userDtoJsonString

[{"id":137,"avatar_url":"https://avatars.githubusercontent.com/u/137?v\u003d4","name":"Joshua Peek","login":"josh","followers":2351,"following":184,"location":"San Francisco, CA"},{"id":54473,"avatar_url":"https://avatars.githubusercontent.com/u/54473?v\u003d4","name":"Josh Long","login":"joshlong","bio":"Josh (@starbuxman) is the Spring Developer Advocate at VMware, an open-source hacker, book/video author and speaker","followers":7955,"following":65,"location":"San Francisco, CA"},{"id":6692932,"avatar_url":"https://avatars.githubusercontent.com/u/6692932?v\u003d4","name":"Joshua Comeau","login":"joshwcomeau","bio":"Software developer.\r\nMakes stuff.","followers":3578,"following":25,"location":"Montreal, QC"},{"id":13760,"avatar_url":"https://avatars.githubusercontent.com/u/13760?v\u003d4","name":"Josh Abernathy","login":"joshaber","followers":2066,"following":39,"location":"The Gem City"},{"id":10632,"avatar_url":"https://avatars.githubusercontent.com/u/10632?v\u003d4","name":"Josh

In [9]:
for (i in 0..9) {
    println(
        """ val fakeUserDto${i+1} = UserDto(
        id = ${listUserDto[i].id},
        avatarUrl = "${listUserDto[i].avatarUrl}",
        name = "${listUserDto[i].name}",
        login = "${listUserDto[i].login}",
        bio = "${listUserDto[i].bio?.trim()?.take(50)?.replace("\r\n", " ")}...",
        followers = ${listUserDto[i].followers},
        following = ${listUserDto[i].following},
        location = "${listUserDto[i].location}",
        email = "${listUserDto[i].email}"
    )""")
}

 val fakeUserDto1 = UserDto(
        id = 137,
        avatarUrl = "https://avatars.githubusercontent.com/u/137?v=4",
        name = "Joshua Peek",
        login = "josh",
        bio = "null...",
        followers = 2351,
        following = 184,
        location = "San Francisco, CA",
        email = "null"
    )
 val fakeUserDto2 = UserDto(
        id = 54473,
        avatarUrl = "https://avatars.githubusercontent.com/u/54473?v=4",
        name = "Josh Long",
        login = "joshlong",
        bio = "Josh (@starbuxman) is the Spring Developer Advocat...",
        followers = 7955,
        following = 65,
        location = "San Francisco, CA",
        email = "null"
    )
 val fakeUserDto3 = UserDto(
        id = 6692932,
        avatarUrl = "https://avatars.githubusercontent.com/u/6692932?v=4",
        name = "Joshua Comeau",
        login = "joshwcomeau",
        bio = "Software developer. Makes stuff....",
        followers = 3578,
        following = 25,
        location = "Mon

In [10]:
for (i in 0..9) {
    println(
        """ val fakeUser${i+1} = User(
        id = ${listOfUser[i].id},
        avatarUrl = "${listOfUser[i].avatarUrl}",
        name = "${listOfUser[i].name}",
        login = "${listOfUser[i].login}",
        bio = "${listOfUser[i].bio.trim().take(50).replace("\r\n", " ")}...",
        followers = ${listOfUser[i].followers},
        following = ${listOfUser[i].following},
        location = "${listOfUser[i].location}",
        email = "${listOfUser[i].email}"
    )""")
}

 val fakeUser1 = User(
        id = 137,
        avatarUrl = "https://avatars.githubusercontent.com/u/137?v=4",
        name = "Joshua Peek",
        login = "josh",
        bio = "Biography not available...",
        followers = 2351,
        following = 184,
        location = "San Francisco, CA",
        email = "N/A"
    )
 val fakeUser2 = User(
        id = 54473,
        avatarUrl = "https://avatars.githubusercontent.com/u/54473?v=4",
        name = "Josh Long",
        login = "joshlong",
        bio = "Josh (@starbuxman) is the Spring Developer Advocat...",
        followers = 7955,
        following = 65,
        location = "San Francisco, CA",
        email = "N/A"
    )
 val fakeUser3 = User(
        id = 6692932,
        avatarUrl = "https://avatars.githubusercontent.com/u/6692932?v=4",
        name = "Joshua Comeau",
        login = "joshwcomeau",
        bio = "Software developer. Makes stuff....",
        followers = 3578,
        following = 25,
        location = "Mont

In [11]:
data class I(
    val name: String
)

val i = I("Saya")

val ii = i.copy(name = "Gue")

println(i.toString())
println(ii.toString())

I(name=Saya)
I(name=Gue)


In [12]:
val userId: Long
val avatarUrl: String?
val name: String?
val login: String?
val followers: Int?
val following: Int?
val location: String?
    
assertEquals(137L, user.userId)
assertEquals("accompanist", user.name)
assertEquals("A collection of extension libraries for Jetpack Co...", user.avatarUrl)
assertEquals(0, user.login)
assertEquals("2022-01-22T12:30:12Z", user.followers)
assertEquals(449630544L, user.following)
assertEquals("2022-01-22T12:30:12Z", user.location)

Line_18.jupyter-kts (1:1 - 17) Property must be initialized or be abstract
Line_18.jupyter-kts (2:1 - 23) Property must be initialized or be abstract
Line_18.jupyter-kts (3:1 - 18) Property must be initialized or be abstract
Line_18.jupyter-kts (4:1 - 19) Property must be initialized or be abstract
Line_18.jupyter-kts (5:1 - 20) Property must be initialized or be abstract
Line_18.jupyter-kts (6:1 - 20) Property must be initialized or be abstract
Line_18.jupyter-kts (7:1 - 22) Property must be initialized or be abstract
Line_18.jupyter-kts (9:1 - 13) Unresolved reference: assertEquals
Line_18.jupyter-kts (9:20 - 24) Unresolved reference: user
Line_18.jupyter-kts (10:1 - 13) Unresolved reference: assertEquals
Line_18.jupyter-kts (10:29 - 33) Unresolved reference: user
Line_18.jupyter-kts (11:1 - 13) Unresolved reference: assertEquals
Line_18.jupyter-kts (11:71 - 75) Unresolved reference: user
Line_18.jupyter-kts (12:1 - 13) Unresolved reference: assertEquals
Line_18.jupyter-kts (12:17 - 